In [1]:
import tensorflow as tf

# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

Num GPUs Available:  1
['/device:CPU:0', '/device:GPU:0']


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

train_dir = 'dataset/Train'
valid_dir = 'dataset/Test'

training_datagen = ImageDataGenerator(
    rescale=1./255.,
	rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    class_mode='categorical',
    batch_size=32
)

validation_generator = validation_datagen.flow_from_directory(
	valid_dir,
	target_size=(224,224),
	class_mode='categorical',
    batch_size=32
)

Found 21044 images belonging to 14 classes.
Found 6738 images belonging to 14 classes.


In [3]:
import tensorflow as tf
import tensorflow_hub as hub

mobilenet_v3 = "https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/classification/5"

feature_extractor_model = mobilenet_v3

feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(14,activation='softmax')
])

model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1001)              5508713   
                                                                 
 dense (Dense)               (None, 512)               513024    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 14)                7182      
                                                                 
Total params: 6,028,919
Trainable params: 520,206
Non-trainable params: 5,508,713
_________________________________________________________________


In [4]:
%load_ext tensorboard
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

In [5]:
import shutil

# Clear any logs from previous runs
shutil.rmtree('./logs/')

In [6]:
NUM_EPOCHS = 10

history = model.fit(train_generator,
                    validation_data=validation_generator,
                    epochs=NUM_EPOCHS,
                    verbose=1,
                    callbacks=[tensorboard_callback,early_stopping])

Epoch 1/10
658/658 [==============================] - 274s 402ms/step - loss: 0.2295 - accuracy: 0.9252 - val_loss: 0.1461 - val_accuracy: 0.9481
Epoch 2/10
658/658 [==============================] - 232s 353ms/step - loss: 0.1150 - accuracy: 0.9604 - val_loss: 0.0985 - val_accuracy: 0.9676
Epoch 3/10
658/658 [==============================] - 230s 349ms/step - loss: 0.0964 - accuracy: 0.9668 - val_loss: 0.0957 - val_accuracy: 0.9665
Epoch 4/10
658/658 [==============================] - 229s 348ms/step - loss: 0.0769 - accuracy: 0.9723 - val_loss: 0.0925 - val_accuracy: 0.9681
Epoch 5/10
658/658 [==============================] - 229s 348ms/step - loss: 0.0839 - accuracy: 0.9714 - val_loss: 0.0699 - val_accuracy: 0.9763
Epoch 6/10
658/658 [==============================] - 234s 355ms/step - loss: 0.0756 - accuracy: 0.9755 - val_loss: 0.0677 - val_accuracy: 0.9767
Epoch 7/10
658/658 [==============================] - 235s 357ms/step - loss: 0.0666 - accuracy: 0.9775 - val_loss: 0.0981 -

In [8]:
#tensorboard
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 5340), started 0:37:52 ago. (Use '!kill 5340' to kill it.)

In [ ]:
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
import matplotlib.pyplot as plt

class_labels = ["Cardboard", "Glass", "Metal", "Organic", "Paper","Plastic"]

# Path to the directory containing your test images
test_dir = os.path.join('data','test')

# Get a list of all image files in the test directory
image_paths = [os.path.join(test_dir, file) for file in os.listdir(test_dir) if file.endswith(('.jpg', '.png', '.jpeg'))]

for image_path in image_paths:
    # Load and preprocess the image using Pillow (PIL)
    img = load_img(image_path, target_size=(224, 224))  # Load and resize the image
    x = img_to_array(img)  # Convert the image to a NumPy array
    x = x / 255.0  # Normalize the pixel values to the range [0, 1]

    # Predict using the model
    yhat = model.predict(np.expand_dims(x, axis=0))
    predicted_class = np.argmax(yhat, axis=1)[0]
    predicted_label = class_labels[predicted_class]

    print(f'Predicted class for {image_path} is {predicted_label}')
    print(f'Class index: {predicted_class}')

    plt.imshow(x)  # Display the resized image
    plt.title(f'Predicted class: {predicted_label}')
    plt.show()

In [10]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('converted_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\ruru\AppData\Local\Temp\tmp_00rb3uj\assets


INFO:tensorflow:Assets written to: C:\Users\ruru\AppData\Local\Temp\tmp_00rb3uj\assets
